In [ ]:
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2
!pip install -e .
 
!pip install pydantic-settings
!pip install peft
!pip install trl
!pip install bitsandbytes
!cd FlagEmbedding/LM_Cocktail/
!pip install -e . 
# Restart the Python process to use the updated packages
dbutils.library.restartPython()

In [ ]:
from pathlib import Path
from Reimer.automatic_merging import create_merging_samples, call_mix_models, call_mix_models_with_data

create samples

In [ ]:
data_dir = "/dbfs/Reimer/data"
samples_dir = "/dbfs:/Reimer/merging_data"
seed=42
N = 20
task="sib200"
language="de"

print(f"generating merging data for {task}-{language} with seed={seed}")
data_path = f"{data_dir}/{task}/{task}_{language}_validation-filtered.json"
samples_path = f"{samples_dir}/{task}/{task}_{language}_{seed}_samples_{N}.json"
dbutils.fs.mkdirs(samples_dir)
samples_dir = samples_dir.replace(":", "")
# create_merging_samples(task=task, language=language, N=N, seed=seed, data_path=data_path, output_path=samples_path, databricks=True)

In [ ]:
data_dir = "/dbfs/Reimer/data"
# samples_dir = "/dbfs:/Reimer/merging_data"
seed=42
N = 20
from Reimer.constants import TRAINING_LANGUAGES

for task in TRAINING_LANGUAGES.keys():
    for language in TRAINING_LANGUAGES[task]:
        print(f"generating merging data for {task}-{language} with seed={seed}")
        data_path = f"{data_dir}/{task}/{task}_{language}_validation.json"
        samples_path = f"{samples_dir}/{task}/{task}_{language}_{seed}_samples_{N}.json"
        dbutils.fs.mkdirs(samples_dir)
        # create_merging_samples(task=task, language=language, N=N, seed=seed, data_path=data_path)

Merging

In [ ]:
def create_model_path(
    base_model,
    task,
    language,
    ft_type,
    base_path = "/dbfs/Reimer/output",
):
    return f"{base_path}/{task}/experiment_{base_model}_{task}_{ft_type}_{language}/"

base-ft, mono and pooling

In [ ]:
examples_dir = Path("/Workspace/Users/rkoopal@deloitte.nl/merging_samples")
examples_paths = [str(p) for p in examples_dir.iterdir() if p.is_file()]
languages = ['ar', 'de', 'el', 'es']

for lang in languages:
    base_path = "/dbfs/Reimer/output"
    model1 = "Bigscience/mt0-large"
    model2 = create_model_path(base_model="mt0-large", task="wikiann", language=lang, ft_type="ft")
    
    # print(f"Mixing (mono) with {model1}, {model2}")
    # call_mix_models(models=[model1, model2], output_dir="/dbfs/Reimer/merged_models/mono", databricks=True)

    # print(f"Mixing (pooling) with {model1}, {model2}")
    # call_mix_models_with_data(models=[model1, model2], output_dir="/dbfs/Reimer/merged_models/pooling", example_paths=examples_paths, databricks=True)

ft-ft - mono

In [ ]:
from itertools import combinations_with_replacement, combinations
languages = ['ar', 'de', 'el', 'es']

for lang1, lang2 in combinations(languages, 2):
    print(f"{lang1} - {lang2}")

In [ ]:
from itertools import product, combinations_with_replacement, combinations
from tqdm import tqdm
examples_dir = Path("/Workspace/Users/rkoopal@deloitte.nl/merging_samples")
examples_paths = [str(p) for p in examples_dir.iterdir() if p.is_file()]
languages = ['ar', 'de', 'el', 'es']

task1 = "wikiann"
task2 = "wikiann"

for lang1, lang2 in tqdm(combinations(languages, 2)):
    model1 = create_model_path(base_model="mt0-large", task=task1, language=lang1, ft_type="ft")
    model2 = create_model_path(base_model="mt0-large", task=task2, language=lang2, ft_type="ft")
    
    print(f"Mixing (mono) with {model1}, {model2}")
    call_mix_models(models=[model1, model2], output_dir="/dbfs/Reimer/merged_models/mono", databricks=True)

    # print(f"Mixing (pooling) with {model1}, {model2}")
    # call_mix_models_with_data(models=[model1, model2], output_dir="/dbfs/Reimer/merged_models/pooling", example_paths=examples_paths, databricks=True)

In [ ]:
display(dbutils.fs.ls("dbfs:/Reimer/merged_models/mono"))